In [ ]:
# Cell 1 - Install packages
!pip install mediapipe==0.10.21 tensorflow==2.12.0 scikit-learn==1.2.2 matplotlib==3.7.1 textblob==0.17.1 opencv-python==4.7.0.72

# Cell 2 - Verify installations
import pkg_resources
for pkg in ['mediapipe', 'tensorflow', 'scikit-learn', 'matplotlib', 'textblob', 'opencv-python']:
    print(f"{pkg}: {pkg_resources.get_distribution(pkg).version}")

# Cell 3 - Restart runtime (Runtime -> Restart runtime) before proceeding

In [ ]:
# After restart, run this cell first to verify everything loaded correctly
import mediapipe
import tensorflow
import sklearn
import matplotlib
import textblob
import cv2
print("All imports working!")

In [ ]:
import os
import shutil
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import mediapipe as mp
import cv2


In [ ]:
# Re-mount Drive to ensure all files are accessible
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)

In [ ]:
# Initializing the Model
mpHands = mp.solutions.hands
hands = mpHands.Hands(
	static_image_mode=True,
	model_complexity=1,
	min_detection_confidence=0.5,
	min_tracking_confidence=0.5,
	max_num_hands=2)

In [ ]:
# Define paths
data_dir = '/content/drive/MyDrive/asl-dataset-v1/asl-dataset-v1/data'

# Define the 26 letter classes we want to keep
valid_classes = [chr(ord('A') + i) for i in range(26)]  # A-Z

# Collect all image file paths and their corresponding class labels
file_paths = []
labels = []

for class_dir in os.listdir(data_dir):
    # Only process directories that are in our valid_classes list
    if class_dir in valid_classes and os.path.isdir(os.path.join(data_dir, class_dir)):
        class_path = os.path.join(data_dir, class_dir)
        for img_file in os.listdir(class_path):
            # Skip hidden files like .DS_Store
            if not img_file.startswith('.'):
                file_paths.append(os.path.join(class_path, img_file))
                labels.append(class_dir)

file_paths = np.array(file_paths)
labels = np.array(labels)
class_names = np.unique(labels)

# Verify we have exactly 26 classes
print("Classes found:", class_names)
assert len(class_names) == 26, f"Expected 26 classes but found {len(class_names)}"


def load_images(file_paths):
    images = []
    valid_labels = []
    count = 0
    for file_path, label in zip(file_paths, labels):

        img = cv2.imread(file_path)
        # Convert BGR image to RGB image
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        # Process the RGB image
        results = hands.process(imgRGB)
        if results.multi_hand_landmarks:
            #Used for data normalization
            x_min = min([landmark.x for landmark in results.multi_hand_landmarks[0].landmark])
            y_min = min([landmark.y for landmark in results.multi_hand_landmarks[0].landmark])
            x_max = max([landmark.x for landmark in results.multi_hand_landmarks[0].landmark])
            y_max = max([landmark.y for landmark in results.multi_hand_landmarks[0].landmark])
            w = x_max - x_min
            h = y_max - y_min
            #Generate normlized data List with its flipped version, and flatten
            landmarks      = [(   (landmark.x - x_min) / w , (landmark.y - y_min) / h ) for landmark in results.multi_hand_landmarks[0].landmark]
            landmarks_Flip = [(1-((landmark.x - x_min) / w), (landmark.y - y_min) / h ) for landmark in results.multi_hand_landmarks[0].landmark]
            landmarks = list(sum(landmarks, ()))
            landmarks_Flip = list(sum(landmarks_Flip, ()))
            #Append to the processed dataset. X(images), y(valid_labels)
            images.append(landmarks)
            valid_labels.append(label)
            images.append(landmarks_Flip)
            valid_labels.append(label)
        else:
            count += 1


    print("Missed = {}".format(count))
    return np.array(images), np.array(valid_labels)
# Load all images
images, valid_labels = load_images(file_paths)

# Create stratified train/test split
train_files, test_files, train_labels, test_labels = train_test_split(
    images, valid_labels, test_size=0.3, stratify=valid_labels, random_state=42
)
# Further split the test set into validation and test sets
val_files, test_files, val_labels, test_labels = train_test_split(
    test_files, test_labels, test_size=0.5, stratify=test_labels, random_state=42
)
encoder = OneHotEncoder(categories='auto', sparse=False)
# Reshape labels for OneHotEncoder
train_labels_reshaped = train_labels.reshape(-1, 1)
val_labels_reshaped = val_labels.reshape(-1, 1)
test_labels_reshaped = test_labels.reshape(-1, 1)
# Fit and transform labels
train_labels_encoded = encoder.fit_transform(train_labels_reshaped)
val_labels_encoded = encoder.transform(val_labels_reshaped)
test_labels_encoded = encoder.transform(test_labels_reshaped)

# Check the shapes of the datasets
print(f"Train files shape: {train_files.shape}")
print(f"Train labels shape: {train_labels_encoded.shape}")
print(f"Validation files shape: {val_files.shape}")
print(f"Validation labels shape: {val_labels_encoded.shape}")
print(f"Test files shape: {test_files.shape}")
print(f"Test labels shape: {test_labels_encoded.shape}")

In [ ]:
# Define a simple CNN model

# Build the model
model = Sequential()
model.add(Dense(64, input_dim=42, activation='relu'))  # First hidden layer with 64 neurons
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(128, activation='relu'))  # Second hidden layer with 128 neurons
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(128, activation='relu'))  # Second hidden layer with 128 neurons
model.add(Dropout(0.5))  # Dropout to prevent overfitting
model.add(Dense(26, activation='softmax'))  # Output layer


# Compile the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Define callbacks
checkpoint = ModelCheckpoint('c:/final_model/model.keras', save_best_only=True, monitor='val_accuracy', mode='max')
early_stopping = EarlyStopping(monitor='val_accuracy', patience=8, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_accuracy', factor=0.2, patience=3, min_lr=0.001)

# Train the model
#steps_per_epoch = len(train_files) // train_generator.batch_size
#validation_steps = len(val_files) // validation_generator.batch_size

history = model.fit(
    train_files, train_labels_encoded,
    batch_size = 64,
    #steps_per_epoch=steps_per_epoch,
    epochs=100,
    validation_data=(val_files, val_labels_encoded),
    #validation_steps=validation_steps,
    callbacks=[checkpoint, early_stopping, reduce_lr]
)

# Evaluate on test data
test_loss, test_acc = model.evaluate(test_files, test_labels_encoded )
print(f'Test accuracy: {test_acc}')

In [ ]:

model.summary()

In [ ]:
from sklearn.metrics import f1_score
import matplotlib.pyplot as plt
import numpy as np

y_pred = model.predict(test_files)
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(test_labels_encoded, axis=1)

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report

accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1_score = f1_score(y_true, y_pred, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)

print(classification_report(y_true, y_pred))




In [ ]:
import seaborn as sns
cm = confusion_matrix(y_true, y_pred)
plt.figure(figsize=(8, 6))
ax = plt.subplot()
sns.heatmap(cm, annot=True, ax=ax)  # Annotate cells with values
ax.set_xlabel('Predicted labels')
ax.set_ylabel('True labels')
ax.set_title('Confusion Matrix')
ax.xaxis.set_ticklabels(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])
ax.yaxis.set_ticklabels(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z'])

plt.show()

In [ ]:
def get_bounding_box(landmarks, image_width, image_height, scale=1.2):
    x_coords = [landmark.x * image_width for landmark in landmarks.landmark]
    y_coords = [landmark.y * image_height for landmark in landmarks.landmark]
    x_min, x_max = int(min(x_coords)), int(max(x_coords))
    y_min, y_max = int(min(y_coords)), int(max(y_coords))

    # Calculate the center of the bounding box
    x_center = (x_min + x_max) // 2
    y_center = (y_min + y_max) // 2

    # Calculate the size of the bounding box
    box_size = max(x_max - x_min, y_max - y_min) * scale

    # Ensure the bounding box is a square
    half_size = int(box_size // 2)

    # Calculate new min and max coordinates
    x_min_new = max(x_center - half_size, 0)
    x_max_new = min(x_center + half_size, image_width)
    y_min_new = max(y_center - half_size, 0)
    y_max_new = min(y_center + half_size, image_height)

    return x_min_new, y_min_new, x_max_new, y_max_new




In [ ]:
from collections import Counter
from textblob import TextBlob

smoothing_window_size = 5
autocorrection_threshold = 3

def smooth_predictions(predictions, window_size = smoothing_window_size):
    smoothed = []
    for i in range(len(predictions) - window_size + 1):
        window = predictions[i:i + window_size]
        most_common = Counter(window).most_common(1)[0][0]
        smoothed.append(most_common)
    return smoothed

def remove_redundant(predictions, threshold=4):
    filtered = []
    last_char = predictions[0]
    count = 0

    for char in predictions:
        if char == last_char:
            count += 1
        else:
            if count >= threshold:
                filtered.append(last_char)
            count = 1
            last_char = char
    if count >= threshold:
            filtered.append(last_char)

    return filtered

def process_predicted_word(letters_list):
    letters_list = letters_list
    #if input is too small -> don't do processing
    if len(letters_list) < smoothing_window_size:
        return ''.join(letters_list).lower()
    else:
        smoothing = smooth_predictions(letters_list)
        filter_redundants= remove_redundant(smoothing)
        if len(filter_redundants) <= autocorrection_threshold:
            return ''.join(filter_redundants).lower()
        autocorrected = str(TextBlob(''.join(filter_redundants).lower()).correct())
        return autocorrected


#FOR DEMONSTRATION

x = ['h','h','h','h','h','x','a','a','a','a','x','x','a','a','a','p','p','p','p','p','y','y','y','y','y','y']
print("input: ",x)
print("input flatten: ",''.join(x))
x = np.array(smooth_predictions(x))
#smooth_predictions(x.tolist())
print("Smoothing: ",''.join(x))
x = np.array(remove_redundant(x))
print("Removing Redundants: ",''.join(x))
x = str(TextBlob(''.join(x)).correct())
print("Autocorrecting: ",x)


'''
x = ['G', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'H', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'A', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'P', 'Y', 'Y', 'B', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'C', 'D']
process_predicted_word(x)
str(TextBlob('hapy').correct())
'''

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
import pickle
import numpy as np

#Save the model architecture to a JSON string
model_json = model.to_json()

# Save the model weights to a numpy array
model_weights = model.get_weights()

# Create a dictionary to store the model architecture and weights
model_dict = {
    'model_json': model_json,
    'model_weights': model_weights
}

# Save the dictionary to a pickle file
with open('c:/final_model/model.pkl', 'wb') as f:
    pickle.dump(model_dict, f)